# Topic Modeling

In [ ]:
# This time, I will import the dependencies
# the first time they are used

## Thought Exercise

Training a topic model with _common sense_

1. Produce a random tf-idf representation of a document

In [ ]:
import numpy as np

topic = {}
# Function zip() returns an iterator of tuples, where the i-th tuple 
# contains the i-th element from each of the argument sequences

# np.random.rand(i) produces an array of i random numbers

tfidf = dict(list(zip('cat dog apple lion NYC love'.split(),\
                      np.random.rand(6))))
# Random tf-idf vector for our single document
tfidf

2. __Manually__ create common-sense weights

In [ ]:
# Now, we multiply the tf-idf vector by the 
# "hand-crafted” weights (notice the subtractions)
topic['petness'] = (
                  .3 * tfidf['cat'] 
                + .3 * tfidf['dog'] 
                +  0 * tfidf['apple'] 
                +  0 * tfidf['lion'] 
                - .2 * tfidf['NYC'] 
                + .2 * tfidf['love'])
topic['animalness'] = (
                  .1 * tfidf['cat'] 
                + .1 * tfidf['dog'] 
                - .1 * tfidf['apple'] 
                + .5 * tfidf['lion'] 
                + .1 * tfidf['NYC'] 
                - .1 * tfidf['love'])
topic['cityness'] = ( 
                   0 * tfidf['cat'] 
                - .1 * tfidf['dog'] 
                + .2 * tfidf['apple'] 
                - .1 * tfidf['lion'] 
                + .5 * tfidf['NYC'] 
                + .1 * tfidf['love'])
topic

Back to the slides

3. Transposing the 6x3 matrix to produce **topic weights for each word**

In [ ]:
word_vector = {}

word_vector['cat'] = [.3*topic['petness'],
                    .1*topic['animalness'],
                    0*topic['cityness']]

word_vector['dog'] = [.3*topic['petness'],
                    .1*topic['animalness'], 
                    -.1*topic['cityness']]

word_vector['apple']= [0*topic['petness'],
                    .1*topic['animalness'],
                    .2*topic['cityness']]

word_vector['lion'] = [0*topic['petness'],
                    .5*topic['animalness'],
                    -.1*topic['cityness']]
word_vector['NYC'] = [-.2*topic['petness'],
                    .1*topic['animalness'],
                    .5*topic['cityness']]
word_vector['love'] = [.2*topic['petness'],
                    -.1*topic['animalness'],
                    .1*topic['cityness']]
word_vector

Back to the slides

## Training a Linear Discriminant Analysis classifier

In [ ]:
import pandas as pd

from nlpia.data.loaders import get_data

# Loading a labeled corpus: spam
sms = get_data('sms-spam')
print(sms[:-10])

# Just setting up the printing properties
pd.options.display.width = 120

In [ ]:
# For display purposes: spam instances have a "!" added to the label
index = ['sms{}{}'.format(i, '!'*j) for (i,j) in \
         zip(range(len(sms)), sms.spam)]
print(index[:20])

In [ ]:
#'!'*0
#'!'*1
#'!'*4

In [ ]:
# Creating a pandas df, using the data and the new index
sms = pd.DataFrame(sms.values, columns=sms.columns, index=index)
sms['spam'] = sms.spam.astype(int)
print(sms)
# len(sms)

In [ ]:
# QUESTION: what am I getting with this sum?
sms.spam.sum()

In [ ]:
from nltk.tokenize.casual import casual_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorising the corpus
tfidf_model = TfidfVectorizer(tokenizer=casual_tokenize)
tfidf_docs = tfidf_model.fit_transform(raw_documents=sms.text).toarray()
# QUESTION: what is the number on the right?
print(tfidf_docs.shape)
print(tfidf_docs)




We have 
* 4837 messages
* 638 positive instances
* 9232 types

That's way too much for a Naïve Bayes' classifier

## Implementing the LDA

We just need the centroids of spam and non-spam, so we implement it 

(keep in mind that sklearn has an [LDA](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html))

In [ ]:
# A mask (or "filter") to select only spam messages
mask = sms.spam.astype(bool).values
print(mask)

In [ ]:
# Computing the spam centroid
spam_centroid = tfidf_docs[mask].mean(axis=0)
# axis=0 tells numpy to compute the mean for each column independently
print(spam_centroid.round(2))
len(spam_centroid)

In [ ]:
# Computing the ham centroid
ham_centroid = tfidf_docs[~mask].mean(axis=0)
print(ham_centroid.round(2))
len(ham_centroid)

In [ ]:
spam_centroid - ham_centroid

In [ ]:
# Computing the centroid difference: "the line between spam and ham"
spamminess_score = tfidf_docs.dot(spam_centroid - ham_centroid)
print(spamminess_score.round(2))
len(spamminess_score)

Not just subtracting. We computed the dot product!

**spamminess_score** is $dis(centroid_{(spam)}, centroid_{(ham)})$

We compute it by projecting each TF-IDF vector onto that line between the centroids using the dot product (those were indeed 4,837 dot products computed at once!)

In [ ]:
# Convert the vector spamminess_score into matrix: 
spamminess_score.reshape(-1,1)
# Because the input to the next function must have shape: (n_samples, n_features)

[MinMaxScaler's fit_transform](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler.fit_transform) will scale each of the features to range [0, 1]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Turning the scores into "probabilities" 
sms['lda_score'] = MinMaxScaler().fit_transform(spamminess_score.reshape(-1,1))

# Turning them into predictions
sms['lda_predict'] = (sms.lda_score > .5).astype(int)

# Displaying them
sms['spam lda_predict lda_score'.split()].round(2).head(6)

In [ ]:
# What is the accuracy of the model?
(1. - (sms.spam - sms.lda_predict).abs().sum() / len(sms)).round(3)

In [ ]:
# Getting a confusion matrix
from pugnlp.stats import Confusion

Confusion(sms['spam lda_predict'.split()])

## Some pre-computed topic vectors

In [ ]:
# Topic vectors
from nlpia.book.examples.ch04_catdog_lsa_3x6x16 import word_topic_vectors
word_topic_vectors.T.round(1)

top0 $\approx$ cityness. It contains the highest values for **apple** and **nyc**. It is the first in the LSA ordering. It has the **highest variance** in the dataset

top1 $\approx$ seems to be the one for **love** (see the last column)

top2 $\approx$ has a lot of **dog** and some **love**

Not too much **love** for **cats** (but they are "anti-city")

(the code to end up with this space is available from the book)

end of the notebook